# Analysis of StackOverflow Survey. Part IV

In this notebook we address the third question, and we build a model to predict job satisfaction for data coders.

The steps of the process are: all steps with substeps
1. 
2. 

In [2]:
# general packages and libraries
import os
import sys
from collections import defaultdict
import importlib

In [3]:
# data manipulation packages
import numpy as np
import pandas as pd

In [4]:
# data visualizations packages
import matplotlib.pyplot as plt
# to render plots in the notebook
%matplotlib inline

import seaborn as sns
# set a theme for seaborn
sns.set_theme()

In [5]:
# numerical, statistical and machine learning packages and libraries
import xgboost as xgb
from scipy import stats

from sklearn import (
    ensemble,
    tree,
)
from sklearn.base import (
    BaseEstimator, 
    TransformerMixin,
)
from sklearn.pipeline import (
    make_pipeline,
    FeatureUnion, 
    Pipeline,
)
from sklearn.feature_selection import (
    SelectKBest, 
    chi2, 
    mutual_info_classif,
)
from sklearn.impute import (
    KNNImputer,
    SimpleImputer,
)
from sklearn.preprocessing import (
    OneHotEncoder, 
    OrdinalEncoder, 
    LabelEncoder,
    StandardScaler,
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    KFold,
    cross_val_score,
)

from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#from sklearn.ensemble import (RandomForestClassifier)

from sklearn.linear_model import (
    SGDClassifier,
    LogisticRegression,
) 
from sklearn.metrics import (
    classification_report,
    r2_score, 
    mean_squared_error,
    auc,
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    roc_curve,
)


In [74]:
# import local module containing the neccessary functions
import utils_functions as uf
#import encoder_module as encm

# forces the interpreter to re-load the module
importlib.reload(uf);

# create a path string
mypath = os.getcwd()

## Formulate the questions

We separate the respondents of the 2020 StackOverflow Developer Survey into data developers
(data scientist or machine learning specialist, data or business analyst, data engineer) and other developers. In what follows we restrict the dataset to the data developers and address the following questions:  
 - What can we tell about the job satisfaction of a data developer? 
 - What factors do influence the job satisfaction? 
 
We build a predictive model for the job satisfaction for data developers. This is a multi-class classification question, where the satisfaction levels are: very dissatisfied, slightly dissatisfied, neither satisfied nor dissatisfied, slightly satisfied, very satisfied.

## Performance metrics - to review at the end

The following performance measures will be used in this project:
1. Cross validation via StratifiedKFold with 10 folds.
2. Confusion matrix, in particular precision, recall and F1 score.
3. The ROC curve and the related AUC score.

# Gather and prepare the data

Upload the data and keep the subset that contains those developers that work in data science related fields. 


## Load the data

In [7]:
# upload the datafile as pandas dataframe
df = pd.read_csv(mypath+'/data/survey20_updated.csv', index_col=[0])
# check for success
df.shape

(64461, 61)

## Remove unnecessary data

In [8]:
# create a copy of the data
df1 = df.copy()

### Keep the developers that work with data

In [9]:
# change each string in DevType column into a list of strings
df1['DevType'] = df1['DevType'].str.split(';')

# check the outcome
df1.DevType.iloc[:3]

Respondent
1    [Developer, desktop or enterprise applications...
2           [Developer, full-stack, Developer, mobile]
3                                                  NaN
Name: DevType, dtype: object

In [10]:
# split a row with multiple choices strings in DevType into rows where
# DevType contains only one choice, the index is replicated 
df1=df1.explode('DevType')

# the new dataframe has many more rows now
df1.shape

(172185, 61)

In [11]:
# check the outcome
df1.DevType.iloc[:5]

Respondent
1    Developer, desktop or enterprise applications
1                            Developer, full-stack
2                            Developer, full-stack
2                                Developer, mobile
3                                              NaN
Name: DevType, dtype: object

In [12]:
# drop the rows with missing values in DevType column
df1.dropna(subset=['DevType'], inplace=True)

# check the outcome
df1.DevType.isnull().sum()

0

In [13]:
# retain only those rows that contain data coders
df1 = df1[df1['DevType'].str.contains('Data ')]

# check for success
df1.DevType.iloc[:5]

Respondent
22                                    Data engineer
25                                    Data engineer
30                         Data or business analyst
36                         Data or business analyst
36    Data scientist or machine learning specialist
Name: DevType, dtype: object

### Retain the developers that are employed

In [14]:
# check the employment types for data coders
df1.Employment.value_counts()

Employed full-time                                      9236
Independent contractor, freelancer, or self-employed    1481
Not employed, but looking for work                       564
Employed part-time                                       469
Name: Employment, dtype: int64

In [15]:
# retain only the employed data developers
df1 = df1[df1['Employment'] != 'Not employed, but looking for work']

# check for success
df1.Employment.value_counts()

Employed full-time                                      9236
Independent contractor, freelancer, or self-employed    1481
Employed part-time                                       469
Name: Employment, dtype: int64

### Retain only the respondents that code professionally

In [16]:
# check the professional status of the employed developers
df1.MainBranch.value_counts()

I am a developer by profession                                                   8207
I am not primarily a developer, but I write code sometimes as part of my work    2275
I am a student who is learning to code                                            296
I used to be a developer by profession, but no longer am                          203
I code primarily as a hobby                                                       163
Name: MainBranch, dtype: int64

In [17]:
# create a list of main branch choices
main_choices = df1.MainBranch.value_counts().index.to_list()
# retain those rows where MainBranch contains the respondents that work professionally with data
df1 = df1[df1.MainBranch.isin(main_choices[:2])]

# check the outcome
df1.MainBranch.value_counts()

I am a developer by profession                                                   8207
I am not primarily a developer, but I write code sometimes as part of my work    2275
Name: MainBranch, dtype: int64

### Drop the rows with missing values in JobSat column

In [18]:
# drop rows with missing JobSat
df1.dropna(subset=['JobSat'], inplace=True)

### Remove unnecessary columns

In [19]:
# list of columns to be removed
cols_del = [
    # personal, demographics  information
    #'Respondent', 
    'MainBranch', 'Employment', 'Hobbyist', 
    'Country','Ethnicity', 'Age',
    'Gender', 'Sexuality', 'Trans', 
    
    # related to ConvertedComp
    'CompFreq', 'CompTotal', 'CurrencyDesc', 'CurrencySymbol',
    
    # questions regarding future activities
    'DatabaseDesireNextYear', 'MiscTechDesireNextYear',
    'CollabToolsDesireNextYear', 'PlatformDesireNextYear',
    'LanguageDesireNextYear', 'WebframeDesireNextYear',
    
    # questions regarding this survey
    'SurveyEase', 'SurveyLength', 'WelcomeChange',
    
    # question regarding participation is StackOverflow
    'SOSites', 'SOComm', 'SOPartFreq',
    'SOVisitFreq', 'SOAccount',

    # columns related to other columns
    'Age1stCode', 'YearsCodePro', 'DevClass', 

    # high cardinality, multiple choices columns, add noise 
    'MiscTechWorkedWith', 'DatabaseWorkedWith', 'CollabToolsWorkedWith',
    'WebframeWorkedWith', 'LanguageWorkedWith',

    # questions not relevant to our goal
    'JobHunt', 'JobHuntResearch', 'Stuck',
    'PurchaseResearch', 'PurchaseWhat', 
    'Stuck', 'PurpleLink',
    'OffTopic', 'OtherComms',
    'JobFactors', 'JobSeek',

    # auxiliary columns
    'DevClass']

In [20]:
# drop all the columns in the list
df1.drop(columns=cols_del, inplace=True)

# check the output
df1.shape

(10372, 16)

#### Comments on feature selection
Since we are left with 16 features only, I will not perform additional feature selection. After experimenting with a couple of feature selection options, such as mutual_info_classif and SelectKBest, I observed that the model doesn't not performed substantially better after applying feature selection.

## Preprocess data

### Replace JobSat categories with numerical code and drop missing values

In [21]:
# use the custom labelling 
df1['JobSat'] = df1['JobSat'].map(uf.JobSat_dict)
# check the outcome
df1['JobSat'].value_counts()

5    3582
4    3079
2    1623
3    1242
1     846
Name: JobSat, dtype: int64

### Update YearsCode column entries and dtype

In [22]:
# replace strings with numerical entries
replace_dict = {'Less than 1 year': '0', 'More than 50 years': '51'}
df1.replace(replace_dict, inplace=True)

# change dtype to numeric
df1['YearsCode'] = pd.to_numeric(df1['YearsCode'])

### Replace multiple choices strings with single choice rows

In [71]:
multi_cols = ['PlatformWorkedWith']

# explode the two columns
for col in multi_cols:
    df1 = uf.explode_col(df1, col)
    
# check the outcome
df1.shape

(37025, 16)

### Review data types and data distribution in columns

In [64]:
# the list of numerical columns
num_cols = df1.select_dtypes(include='float64').columns.to_list()
print(num_cols)

['ConvertedComp', 'WorkWeekHrs', 'YearsCode']


In [65]:
# the list of all categorical columns
cat_cols = df1.select_dtypes(include='object').columns.to_list()
print(cat_cols)

['DevType', 'EdLevel', 'DevOps', 'DevOpsImpt', 'EdImpt', 'Learn', 'OnboardGood', 'Overtime', 'OpSys', 'OrgSize', 'PlatformWorkedWith', 'UndergradMajor']


In [66]:
# further divide the categorical columns in
multi_cols = ['PlatformWorkedWith']
uni_cols = list(set(cat_cols) - set(multi_cols))
print(uni_cols)

['DevOpsImpt', 'OnboardGood', 'OpSys', 'EdLevel', 'EdImpt', 'UndergradMajor', 'DevType', 'Learn', 'DevOps', 'OrgSize', 'Overtime']


In [67]:
# for each categorical column, print possible row values and their counts
def list_answers(df, cat_cols):
    for col in cat_cols:
        print(col)
        print(' ')
        print(df1[col].value_counts())
        print(' ')
# print counts and values
#list_answers(df1, all_cat_cols)

Comments:  

All the categorical columns have unique strings as entries. So, at this point we can use one of the encoding methods from sklearn.

### Remove duplicates

In [68]:
# drop duplicate rows, if any
df1.drop_duplicates(subset=None, keep='first', inplace=True)
df1.shape

(37025, 16)

## Refactor code

Rewrite all of the steps for data pre-processing in a single function.

In [75]:
# create a fresh copy of the dataset
dft=df.copy()

In [76]:
# all data cleaning and preprocessing steps
dft = uf.remove_clean_data(dft)
# check the outcome
dft.shape

(37025, 16)

## Sample data, create features and target datasets

Create a dataframe X of features and a pandas series y that contains the labels.

In [30]:
# create a copy of the pre-processed dataframe
df2 = df1.copy()

In [31]:
# create the predictors dataframe
X = df2.drop(columns = 'JobSat')

# create the labels
y = df2['JobSat']

# check for success
X.shape, len(y)

((10372, 15), 10372)

### Isolate a test set

In [77]:
# split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# summarize the data
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (7260, 15) (7260,)
Test (3112, 15) (3112,)


## Impute the missing values

In [78]:
# take the numerical columns in the train dataset
X_train_num = X_train[num_cols]
# create an instance of the KNN imputer
num_imputer = KNNImputer(n_neighbors=5)
# fit_transform the imputer on the training set
X_train_num_imp = pd.DataFrame(num_imputer.fit_transform(X_train_num), 
                               columns=X_train_num.columns)
# separate the numerical columns in the test set
X_test_num = X_test[num_cols]
# transform the test set with the imputer that was fit on the training set
X_test_num_imp = pd.DataFrame(num_imputer.transform(X_test_num), columns=X_test_num.columns)

In [79]:
# create an instance of the scaler
scaler = StandardScaler()

# scale the numerical variables, fit and transform on the straining set
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_num_imp), 
                                columns=X_train_num_imp.columns)
# use the scaler fit on training set to transform the test set
X_test_scaled = pd.DataFrame(scaler.transform(X_test_num_imp), columns=X_test_num_imp.columns)
     

In [80]:
# separate all the categorical columns in the training set
X_train_cat = X_train[cat_cols]
# create an instance of the imputer
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
# fit and transform the training data
X_train_cat_imp = pd.DataFrame(cat_imputer.fit_transform(X_train_cat), 
                               columns=X_train_cat.columns)
# separate the categorical columns in the test set
X_test_cat = X_test[cat_cols]
# transform the test data with the imputer fit on the training set
X_test_cat_imp=pd.DataFrame(cat_imputer.transform(X_test_cat), columns=X_test_cat.columns)

## Encode the predictors

In [81]:
# encode the low cardinality columns
def ord_encode_predictors(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = pd.DataFrame(oe.transform(X_train))
    X_test_enc = pd.DataFrame(oe.transform(X_test))
    return X_train_enc, X_test_enc

In [82]:
# the low cardinality encoded features
X_train_uni_enc, X_test_uni_enc = ord_encode_predictors(X_train_cat_imp[uni_cols],
                                                        X_test_cat_imp[uni_cols])

In [83]:
# encode the high cardinality columns
def encode_predictors(X_train, X_test):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(X_train)
    X_train_enc = pd.DataFrame(enc.transform(X_train))
    X_test_enc = pd.DataFrame(enc.transform(X_test))
    return X_train_enc, X_test_enc

In [84]:
# the high cardinality encoded features
X_train_multi_enc, X_test_multi_enc = encode_predictors(X_train_cat_imp[multi_cols], 
                                                        X_test_cat_imp[multi_cols])

In [85]:
# encode the target variable 
def encode_target(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

In [83]:
# encode the target variable - not necessary
# y_train_enc, y_test_enc = encode_targets(y_train,y_test)

In [85]:
# combine the  X frames 
X_train_cat_enc = pd.concat([X_train_multi_enc, X_train_uni_enc], axis=1)
X_train_prep = pd.concat([X_train_cat_enc, X_train_scaled], axis=1)

X_test_cat_enc = pd.concat([X_test_multi_enc, X_test_uni_enc], axis=1)
X_test_prep = pd.concat([X_test_cat_enc, X_test_scaled], axis=1)


### Create a profiling report

In [44]:
# run this once to generate a profiling report and save it as html file

#import pandas_profiling
#profile = pandas_profiling.ProfileReport(X_train, minimal=False)
#profile.to_file(output_file="data_train_report.html")

## Refactor the code: build processing data pipeline

In [86]:
## refactor code: processing data

# the steps in the categorical pipeline for columns of low cardinality
uni_cat_pipeline = Pipeline( steps = [( 'unicat_selector', uf.FeatureSelector(uni_cols) ),
                                  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                  ( 'ordinal_encoder', OrdinalEncoder() ) ] )

# the steps in the categorical pipeline for columns of high cardinality
multi_cat_pipeline = Pipeline( steps = [( 'multicat_selector', uf.FeatureSelector(multi_cols) ),
                                  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                  ( 'one_hot_encoder', OneHotEncoder(sparse=False) ) ] )
    
# the steps in the numerical pipeline     
num_pipeline = Pipeline( steps = [ ('num_selector', uf.FeatureSelector(num_cols) ),
                                  ('imputer', KNNImputer(n_neighbors=5) ),
                                  ( 'std_scaler', StandardScaler() ) ] )

# combine the numerical and the categorical pipelines
full_pipeline = FeatureUnion( transformer_list = [ ( 'unicat_pipeline', uni_cat_pipeline ), 
                                                  ( 'multicat_pipeline', multi_cat_pipeline ) ,
                                                 ( 'numerical_pipeline', num_pipeline )] )

# Baseline model

In [87]:
# the full pipeline as a step in another pipeline with an estimator as the final step
full_pipeline_m = Pipeline( steps = [ ( 'full_pipeline', full_pipeline),
                                  ( 'model', KNeighborsClassifier(n_neighbors=5) ) ] )

# call fit on it just like any other pipeline
full_pipeline_m.fit( X_train, y_train )

# predict with it like any other pipeline
y_pred = full_pipeline_m.predict( X_test ) 

ValueError: Found unknown categories ['Google Cloud Platform;IBM Cloud or Watson;Linux;Raspberry Pi;Windows', 'Android;AWS;Docker;Google Cloud Platform;IBM Cloud or Watson;Linux;MacOS;Windows', 'Android;Google Cloud Platform;Heroku', 'Android;AWS;Docker;Linux;MacOS;Raspberry Pi;Slack Apps and Integrations;Windows;WordPress', 'Docker;Kubernetes;Linux;Windows;WordPress', 'MacOS;Raspberry Pi;Windows;WordPress', 'Android;AWS;Docker;iOS;Kubernetes;Microsoft Azure;Windows', 'Google Cloud Platform;Linux;Microsoft Azure', 'Docker;Google Cloud Platform;Linux;MacOS;Microsoft Azure', 'Android;Google Cloud Platform;Heroku;Linux;Windows;WordPress', 'Docker;Linux;MacOS;Microsoft Azure;Slack Apps and Integrations', 'Android;Arduino;Docker;Google Cloud Platform;iOS;Linux;MacOS;Microsoft Azure;Raspberry Pi;Windows', 'Android;IBM Cloud or Watson;Linux;Windows', 'Android;AWS;Docker;Windows', 'AWS;iOS;Linux;Windows', 'Arduino;Google Cloud Platform;Linux;Raspberry Pi;WordPress', 'AWS;Heroku;iOS;Linux;MacOS

In [93]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)
         

Confusion Matrix:
[[ 3980    15    11    35    49]
 [   18  7035    40    88    68]
 [   13    35  6167    55    41]
 [   20    69    73 13137   127]
 [   29    82    70   138 17796]]

Classification Report:
              precision    recall  f1-score   support

           1       0.98      0.97      0.98      4090
           2       0.97      0.97      0.97      7249
           3       0.97      0.98      0.97      6311
           4       0.98      0.98      0.98     13426
           5       0.98      0.98      0.98     18115

    accuracy                           0.98     49191
   macro avg       0.98      0.98      0.98     49191
weighted avg       0.98      0.98      0.98     49191

Accuracy: 0.978


In [94]:
full_pipeline_m.classes_

array([1, 2, 3, 4, 5])

In [96]:
def get_performance(model, X, ground_y):
    """Calculate some importance metrics for model evaluation: roc_auc_ovr, accuracy, precision_macro, recall_macro,
    confusion matrix"""
    ground_y = np.squeeze(ground_y)

    predict_y = model.predict(X)
    predict_y_proba = model.predict_proba(X)

    roc_auc_score_perf = roc_auc_score(ground_y, predict_y_proba, average='macro', multi_class='ovr')  # ROC-AUC
    logLoss_perf = log_loss(ground_y, predict_y_proba)

    accuracy_perf = (predict_y == ground_y).sum() / len(predict_y)
    precision_score_perf = precision_score(ground_y, predict_y, average='macro')
    recall_score_perf = recall_score(ground_y, predict_y, average='macro')

    # Confusion matrix:
    # print("Confusion matrix [[TN, FP]\n[FN, TP]]:\n", confusion_matrix(ground_y, predict_y))
    conf_m = confusion_matrix(ground_y, predict_y)

    return roc_auc_score_perf, logLoss_perf, accuracy_perf, precision_score_perf, recall_score_perf, conf_m

In [98]:
roc_auc_score_perf_train, logLoss_perf_train, \
accuracy_perf_train, precision_score_perf_train, recall_score_perf_train, \
conf_m_train = get_performance(full_pipeline_m, X_train, y_train)

train_performance = pd.Series([roc_auc_score_perf_train, logLoss_perf_train, \
                               accuracy_perf_train, 
                               precision_score_perf_train, recall_score_perf_train], 
                              index=['roc-auc_macro', 'logloss', 'accuracy',
                                     'precison_macro', 'recall_macro'])


roc_auc_score_perf_test, logLoss_perf_test, \
accuracy_perf_test, precision_score_perf_test, recall_score_perf_test, \
conf_m_test = get_performance(full_pipeline_m, X_test, y_test)

test_performance = pd.Series([roc_auc_score_perf_test, logLoss_perf_test, \
                               accuracy_perf_test, 
                               precision_score_perf_test, recall_score_perf_test], index=['roc-auc_macro', 'logloss', 'accuracy', 'precison_macro', 'recall_macro'])



performance_check = pd.DataFrame.from_dict({'train': train_performance, 'test': test_performance})
performance_check

NameError: name 'log_loss' is not defined

In [90]:
# the full pipeline as a step in another pipeline with an estimator as the final step
full_pipeline_xgb = Pipeline( steps = [ ( 'full_pipeline', full_pipeline),
                                  ( 'model', xgb.XGBClassifier(objective = 'binary:logistic') ) ] )

# call fit on it just like any other pipeline
full_pipeline_m.fit( X_train, y_train )

# predict with it like any other pipeline
y_pred = full_pipeline_m.predict( X_test ) 

In [61]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)

Confusion Matrix:
[[  87   26    9  280  511]
 [   5  217    8  661  703]
 [   1   42  176  528  632]
 [   1   71   12 1423 1649]
 [   1   68   16  619 3362]]

Classification Report:
              precision    recall  f1-score   support

           1       0.92      0.10      0.17       913
           2       0.51      0.14      0.22      1594
           3       0.80      0.13      0.22      1379
           4       0.41      0.45      0.43      3156
           5       0.49      0.83      0.62      4066

    accuracy                           0.47     11108
   macro avg       0.62      0.33      0.33     11108
weighted avg       0.54      0.47      0.42     11108

Accuracy: 0.474


In [62]:
for model in [DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, 
              RandomForestClassifier, SGDClassifier]:
    make_pipeline(model())
    classifier = model()
    kfold = model_selection.KFold(n_splits=5)
    classifier.fit(X_train_prep, y_train)
    s = model_selection.cross_val_score(classifier, X_test_prep,y_test, cv=kfold)
    #result2 = classification_report(y_test, y_pred, zero_division=0)
    #s = model_selection.cross_val_score(cls, X, y, cv=kfold)
    print(f"{model.__name__:22}  CV_Mean:" f"{s.mean():.3f} CV_STD: {s.std():.2f}")

DecisionTreeClassifier  CV_Mean:0.755 CV_STD: 0.00
KNeighborsClassifier    CV_Mean:0.607 CV_STD: 0.00
GaussianNB              CV_Mean:0.214 CV_STD: 0.03
SVC                     CV_Mean:0.535 CV_STD: 0.01
RandomForestClassifier  CV_Mean:0.818 CV_STD: 0.01
SGDClassifier           CV_Mean:0.364 CV_STD: 0.01


In [80]:
clf_xgb = RandomForestClassifier()
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.59),
              'subsample': stats.uniform(0.3, 0.6),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.4),
              'min_child_weight': [1, 2, 3, 4]
             }

numFolds = 5
n = X_train_prep.shape[0]
kfold_5 = KFold(n, True, 5)

clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

In [81]:
def hyperparameter_tune(base_model, parameters, n_iter, kfold, X=X_train, y=y_train):
    start_time = time.time()
    
    # Arrange data into folds with approx equal proportion of classes within each fold
    k = StratifiedKFold(n_splits=kfold, shuffle=False)
    
    optimal_model = RandomizedSearchCV(base_model,
                            param_distributions=parameters,
                            n_iter=n_iter,
                            cv=k,
                            n_jobs=-1,
                            random_state=42)
    optimal_model.fit(X, y)
    
    stop_time = time.time()

    scores = cross_val_score(optimal_model, X, y, cv=k, scoring="accuracy")
    
    print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time)))
    print("====================")
    print("Cross Val Mean: {:.3f}, Cross Val Stdev: {:.3f}".format(scores.mean(), scores.std()))
    print("Best Score: {:.3f}".format(optimal_model.best_score_))
    print("Best Parameters: {}".format(optimal_model.best_params_))
    
    return optimal_model.best_params_, optimal_model.best_score_

In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint
import matplotlib.pyplot as plt
import time
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [83]:
base_model = RandomForestClassifier(n_jobs=-1,
                                   random_state=42)

lots_of_parameters = {
    "max_depth": [3, 5, 10, None],
    "n_estimators": [100, 200, 300, 400, 500],
    "max_features": randint(1, 3),
    "criterion": ["gini", "entropy"],
    "bootstrap": [True, False],
    "min_samples_leaf": randint(1, 4)
}

parameters = {
    "max_depth": [3, 5, 10, None],
    "n_estimators": [100, 200, 300, 400, 500]
}

best_params, best_score = hyperparameter_tune(base_model, parameters, 10, 5, X_train_prep, y_train)

Elapsed Time: 00:00:38
Cross Val Mean: 0.931, Cross Val Stdev: 0.005
Best Score: 0.930
Best Parameters: {'n_estimators': 200, 'max_depth': None}


In [84]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train_prep, y_train)

RandomForestClassifier(n_estimators=200)

In [ ]:
import shap
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test_prep)

In [ ]:
shap.summary_plot(shap_values, X_test_prep, plot_type="bar")

In [139]:
df_f_type = df.dtypes

In [143]:
df_f_type = df_f_type.loc[~df_f_type.index.isin(['JobSat'])]

In [144]:
cols_if_num = df_f_type.apply(lambda x: np.issubdtype(x, np.number))

In [146]:
cols_if_num.sum()

4

In [147]:
cols_cat = cols_if_num[~cols_if_num].index.tolist()

In [157]:
for col_cat in cols_cat:
        df_sample[col_cat] = df_sample[col_cat].fillna('missing')

In [160]:
cols_num = cols_if_num[cols_if_num].index.tolist()
cols_num

['Age', 'CompTotal', 'ConvertedComp', 'WorkWeekHrs']

In [161]:
for col_num in cols_num:
        df_sample[col_num] = df_sample[col_num].fillna(df[col_num].mean())

In [162]:
df_sample[cols_num].isnull().sum()

Age              0
CompTotal        0
ConvertedComp    0
WorkWeekHrs      0
dtype: int64

In [159]:
enc = OrdinalEncoder()
df_sample[cols_cat] = enc.fit_transform(df_sample[cols_cat])

In [163]:
enc = OrdinalEncoder()
df_sample.loc[:, 'JobSat'] = enc.fit_transform(df_sample[['JobSat']])

In [166]:
df_sample['JobSat']

Respondent
18657    0.0
20529    0.0
32963    0.0
43959    0.0
1912     0.0
        ... 
32979    4.0
47032    4.0
5262     4.0
37419    4.0
6479     4.0
Name: JobSat, Length: 13558, dtype: float64